1. 确定基本的变量（即data sheet）

In [21]:
# scheme O
road_length__O = 5 # km
average_speed__O = 22 # km/h
AADT__O = 15000 # vehicles/day

# scheme A
road_length__A = 9 # km
average_speed__A = 65 # km/h
average_speed__A_O = 28 # km/h
AADT__A = 13000 # vehicles/day
AADT__A_O = 4000 # vehicles/day
construction_cost__A = 90000000 # pounds
maintainance_cost_A = 10000 # pounds/year

# scheme B
road_length_B = 11 # km
average_speed__B = 65 # km/h
average_speed__B_O = 28 # km/h
AADT__B = 13500 # vehicles/day
AADT__B_O = 3500 # vehicles/day
construction_cost__B = 77000000 # pounds
maintenance_cost__B = 10000 # pounds/year

# low growth scenario
growth_rate__low = 0.01 # per year

# high growth scenario
growth_rate__high = 0.02 # per year


计算不同年份的AADT，存储成一个字典

In [22]:
def calculate_aadt(initial_aadt, growth_rate_low, growth_rate_high, start_year, end_year, change_years, is_special_case=False):
    aadt_dict_low = {}
    aadt_dict_high = {}
    for year in range(start_year, end_year + 1):
        if year - start_year < change_years:
            growth_years = year - start_year
        else:
            growth_years = change_years - 1
        if is_special_case:
            aadt_dict_low[year] = initial_aadt * (1 + growth_rate_low) ** (growth_years + 1)
            aadt_dict_high[year] = initial_aadt * (1 + growth_rate_high) ** (growth_years +1)
        else:
            aadt_dict_low[year] = initial_aadt * (1 + growth_rate_low) ** growth_years
            aadt_dict_high[year] = initial_aadt * (1 + growth_rate_high) ** growth_years    
    
    return aadt_dict_low, aadt_dict_high

# 现在，你可以使用这个函数来替代原有的重复代码
# 例如，对于AADT__O的计算：
AADT__O_dict_low, AADT__O_dict_high = calculate_aadt(AADT__O, growth_rate__low, growth_rate__high, 2031, 2090, 20, True)
AADT__A_dict_low, AADT__A_dict_high = calculate_aadt(AADT__A, growth_rate__low, growth_rate__high, 2031, 2090, 20)
AADT__A_O_dict_low, AADT__A_O_dict_high = calculate_aadt(AADT__A_O, growth_rate__low, growth_rate__high, 2031, 2090, 20)
AADT__B_dict_low, AADT__B_dict_high = calculate_aadt(AADT__B, growth_rate__low, growth_rate__high, 2031, 2090, 20)
AADT__B_O_dict_low, AADT__B_O_dict_high = calculate_aadt(AADT__B_O, growth_rate__low, growth_rate__high, 2031, 2090, 20)

print(AADT__O_dict_low[2090])

18302.850599219506


计算GC中旅行时间节省的部分；
这里工作和非工作的数值是一致的，故不区分

In [23]:
the_all_week_value_of_time_for_an_average_car = 10.79 # pounds/h
the_coefficient_of_converting_time_to_2030 = 1.5893 # using the GDP deflator
Income_growth_previous_efficiency = 125.57 / 104.88 # using the annual parameters in WebTAG
the_value_of_time_2030 = the_all_week_value_of_time_for_an_average_car * the_coefficient_of_converting_time_to_2030 * Income_growth_previous_efficiency
long_term_growth_rate = 0.015

# 定义计算旅行时间价值的函数
def calculate_value_of_time(start_year, end_year, initial_value, growth_rate):
    value_of_time_dict = {}
    for year in range(start_year, end_year + 1):
        value_of_time_dict[year] = initial_value * (1 + growth_rate) ** (year - start_year + 1)
    return value_of_time_dict

# 定义计算旅行时间感知成本的函数
def calculate_journey_time_cost(value_of_time_dict, road_length, average_speed):
    journey_time_cost = {}
    for year, value_of_time in value_of_time_dict.items():
        journey_time_cost[year] = value_of_time * road_length / average_speed
    return journey_time_cost

# 计算2031年到2090年每年的旅行时间价值
the_value_of_time_dict = calculate_value_of_time(2031, 2091, the_value_of_time_2030, long_term_growth_rate)

# 使用计算旅行时间感知成本的函数计算不同路线的成本
GC_for_journey_time_O = calculate_journey_time_cost(the_value_of_time_dict, road_length__O, average_speed__O)
GC_for_journey_time_A = calculate_journey_time_cost(the_value_of_time_dict, road_length__A, average_speed__A)
GC_for_journey_time_A_O = calculate_journey_time_cost(the_value_of_time_dict, road_length__O, average_speed__A_O)
GC_for_journey_time_B = calculate_journey_time_cost(the_value_of_time_dict, road_length_B, average_speed__B)
GC_for_journey_time_B_O = calculate_journey_time_cost(the_value_of_time_dict, road_length__O, average_speed__B_O)

GC_for_journey_time_A[2090]

6.945639388876915

计算GC中的燃料成本和非燃料成本，注意非工作非燃料成本不用半数法(先不考虑)，而非工作燃料成本其实等于工作燃料成本

In [24]:
import csv

# 初始化字典
data_dict = {}

with open('voc.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # 跳过标题行
    for row in reader:
        year = int(row[0])
        data_dict[year] = {
            'fuel_price_per_kwh': float(row[1]),
            'fuel_efficiency': float(row[2]),
            'emission_factor': float(row[3]),
            'money_value_of_emission': float(row[4]),
            'petrol_fuel_price_65': float(row[5]),
            'petrol_fuel_price_28': float(row[6]),
            'petrol_fuel_price_22': float(row[7]),
            'non_work_petrol_fuel_price_65': float(row[8]),
            'non_work_petrol_fuel_price_28': float(row[9]),
            'non_work_petrol_fuel_price_22': float(row[10]),
        }

def calculate_fuel_cost(year, road_length):
    return data_dict[year]['fuel_price_per_kwh'] * data_dict[year]['fuel_efficiency'] * road_length * 1.5893 

def get_non_fuel_efficiency(type, velocity):
    # 1代表work, 2代表non-work
    return 1.157 + (135.946 / velocity if type == '1' else 0)

def calculate_non_fuel_cost(velocity, road_length, type):
    return get_non_fuel_efficiency(type, velocity) * road_length * 1.5893


GC_for_fuel_cost_O = {}
GC_for_fuel_cost_A = {}
GC_for_fuel_cost_B = {}
GC_for_fuel_cost_A_O = {}
GC_for_fuel_cost_B_O = {}

# 以“O”路线的计算为例，其他路线类似
for year in range(2031, 2091):
    GC_for_fuel_cost_A[year] = calculate_fuel_cost(year, road_length__A)


GC_for_non_fuel_work_O = calculate_non_fuel_cost(average_speed__O, road_length__O, '1')
GC_for_non_fuel_work_A = calculate_non_fuel_cost(average_speed__A, road_length__A, '1')
GC_for_non_fuel_work_A_O = calculate_non_fuel_cost(average_speed__A_O, road_length__O, '1')
GC_for_non_fuel_work_B = calculate_non_fuel_cost(average_speed__B, road_length_B, '1')
GC_for_non_fuel_work_B_O = calculate_non_fuel_cost(average_speed__B_O, road_length__O, '1')

GC_O_work = {}
GC_A_work = {}
GC_A_O_work = {}
GC_B_work = {}
GC_B_O_work = {}

for year in range(2031, 2091):
    GC_A_work[year] = GC_for_journey_time_A[year] + GC_for_fuel_cost_A[year] / 100 + GC_for_non_fuel_work_A / 100
    
GC_for_fuel_cost_A[2090]

    

33.22273362004345

In [25]:
import csv

# 初始化字典
fuel_price_per_kwh = {}
fuel_effieciency = {}
emission_factor = {}
money_value_of_emission = {}
petrol_fuel_price_65 = {}
petrol_fuel_price_28 = {}
petrol_fuel_price_22 = {}
non_work_petrol_fuel_price_65 = {} 
non_work_petrol_fuel_price_28 = {}
non_work_petrol_fuel_price_22 = {}

with open('voc.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)  # 跳过标题行
    for rows in reader:
        year = int(rows[0])
        fuel_price_per_kwh[year] = float(rows[1])
        fuel_effieciency[year] = float(rows[2])
        emission_factor[year] = float(rows[3])
        money_value_of_emission[year] = float(rows[4])
        petrol_fuel_price_65[year] = float(rows[5])
        petrol_fuel_price_28[year] = float(rows[6])
        petrol_fuel_price_22[year] = float(rows[7])
        non_work_petrol_fuel_price_65[year] = float(rows[8])
        non_work_petrol_fuel_price_28[year] = float(rows[9])
        non_work_petrol_fuel_price_22[year] = float(rows[10])

GC_for_fuel_cost_O = {} 
GC_for_fuel_cost_A = {}
GC_for_fuel_cost_B = {}

for year in range(2031, 2091):
    GC_for_fuel_cost_O[year] = fuel_price_per_kwh[year] * fuel_effieciency[year] * road_length__O * 1.5893
    GC_for_fuel_cost_A[year] = fuel_price_per_kwh[year] * fuel_effieciency[year] * road_length__A * 1.5893
    GC_for_fuel_cost_B[year] = fuel_price_per_kwh[year] * fuel_effieciency[year] * road_length_B * 1.5893

def get_non_fuel_efficiency(type, velocity):
    if type == '1':
        return 1.157 + 135.946 / velocity
    elif type == '0':
        return 1.157 

GC_for_non_fuel_non_Work_O = get_non_fuel_efficiency('0', average_speed__O) * road_length__O * 1.5893
GC_for_non_fuel_non_Work_A = get_non_fuel_efficiency('0', average_speed__A) * road_length__A * 1.5893
GC_for_non_fuel_non_Work_A_O = get_non_fuel_efficiency('0', average_speed__A_O) * road_length__O * 1.5893
GC_for_non_fuel_non_Work_B = get_non_fuel_efficiency('0', average_speed__B) * road_length_B * 1.5893
GC_for_non_fuel_non_Work_B_O = get_non_fuel_efficiency('0', average_speed__B_O) * road_length__O * 1.5893
GC_for_non_fuel_work_O = get_non_fuel_efficiency('1', average_speed__O) * road_length__O* 1.5893
GC_for_non_fuel_work_A = get_non_fuel_efficiency('1', average_speed__A) * road_length__A * 1.5893
GC_for_non_fuel_work_A_O = get_non_fuel_efficiency('1', average_speed__A_O) * road_length__O * 1.5893
GC_for_non_fuel_work_B = get_non_fuel_efficiency('1', average_speed__B) * road_length_B * 1.5893
GC_for_non_fuel_work_B_O = get_non_fuel_efficiency('1', average_speed__B_O) * road_length__O * 1.5893

GC_O_work = {}
GC_A_work = {}
GC_A_O_work = {}
GC_B_work = {}
GC_B_O_work = {}

for year in range(2031, 2091):
    GC_O_work[year] = GC_for_journey_time_O[year] + GC_for_fuel_cost_O[year] / 100 + GC_for_non_fuel_work_O /100
    GC_A_work[year] = GC_for_journey_time_A[year] + GC_for_fuel_cost_A[year] / 100 + GC_for_non_fuel_work_A /100
    GC_A_O_work[year] = GC_for_journey_time_A_O[year] + GC_for_fuel_cost_A[year] / 100 + GC_for_non_fuel_work_A_O /100
    GC_B_work[year] = GC_for_journey_time_B[year] + GC_for_fuel_cost_B[year] / 100 + GC_for_non_fuel_work_B /100
    GC_B_O_work[year] = GC_for_journey_time_B_O[year] + GC_for_fuel_cost_B[year]/100 + GC_for_non_fuel_work_B_O/100

GC_for_non_fuel_non_Work_A
    




16.5493809

先只计算工作部分, 计算结果以英镑为单位

In [26]:
# 根据ROH的公式计算user benefit
def get_ROH(GC_new, GC_new_old, GC_old, AADT_new, AADT_new_old, AADT_old, ratio):
    '''
    GC_new: GC_A(B)
    GC_new_old: GC_A(B)_O
    GC_old: GC_O
    AADT_new: AADT_A(B)
    AADT_new_old: AADT_A(B)_O
    AADT_old: AADT_O
    '''
    GC_average = (GC_new * AADT_new + GC_new_old * AADT_new_old) / (AADT_new + AADT_new_old)
    return (GC_old - GC_average) * (AADT_new + AADT_new_old + AADT_old) / 2 * 365.25

user_benefit_A_work_low = {}
user_benefit_A_work_high = {}
user_benefit_B_work_low = {}
user_benefit_B_work_high = {}


for year in range(2031, 2091):
    user_benefit_A_work_low[year] = get_ROH(GC_A_work[year], GC_A_O_work[year], GC_O_work[year], 0.053 * AADT__A_dict_low[year], 0.053 * AADT__A_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    user_benefit_A_work_high[year] = get_ROH(GC_A_work[year], GC_A_O_work[year], GC_O_work[year], 0.053 * AADT__A_dict_high[year], 0.053 * AADT__A_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    user_benefit_B_work_low[year] = get_ROH(GC_B_work[year], GC_B_O_work[year], GC_O_work[year], 0.053 * AADT__B_dict_low[year], 0.053 * AADT__B_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    user_benefit_B_work_high[year] = get_ROH(GC_B_work[year], GC_B_O_work[year], GC_O_work[year], 0.053 * AADT__B_dict_high[year], 0.053 * AADT__B_O_dict_high[year], 0.053 * AADT__O_dict_high[year])



下面这部分考虑的是将未来收益折现到2030年

In [27]:
def discount_to_2030(values, end_year):
    """
    对给定的每年收益按年折现到2030年。
    对于距2030年30年内的年份使用3.5%的折现率，超过30年的部分使用3%的折现率。
    
    参数:
    values: 一个字典，键为年份，值为那一年的收益。
    
    返回:
    2030年的折现总值。
    """
    discount_value = 0  # 初始化折现后的总值
    for year, value in values.items():
        if year > end_year:
            continue

        years_diff = year - 2030  # 计算与2030年的年差
        if years_diff <= 30:
            # 对于30年及以内的部分，使用3.5%的折现率
            discount_factor = (1 + 0.035) ** years_diff
        else:
            # 对于超过30年的部分，前30年使用3.5%，之后使用3%
            discount_factor_30 = (1 + 0.035) ** 30
            discount_factor_after_30 = (1 + 0.03) ** (years_diff - 30)
            discount_factor = discount_factor_30 * discount_factor_after_30
        
        discount_value += value / discount_factor  # 将该年的收益折现到2030年的价值，并累加到总值中
    
    return discount_value

print(discount_to_2030(user_benefit_A_work_high, 2060))

14224945.19950175


下面考虑将非工作燃料成本的ROH

In [28]:
GC_O_non_work_fuel = {}
GC_A_non_work_fuel = {}
GC_A_O_non_work_fuel = {}
GC_B_non_work_fuel = {}
GC_B_O_non_work_fuel = {}

for year in range(2031, 2091):
    GC_O_non_work_fuel[year] = GC_for_journey_time_O[year] + GC_for_fuel_cost_O[year] / 100
    GC_A_non_work_fuel[year] = GC_for_journey_time_A[year] + GC_for_fuel_cost_A[year] / 100
    GC_A_O_non_work_fuel[year] = GC_for_journey_time_A_O[year] + GC_for_fuel_cost_A[year] / 100
    GC_B_non_work_fuel[year] = GC_for_journey_time_B[year] + GC_for_fuel_cost_B[year] / 100
    GC_B_O_non_work_fuel[year] = GC_for_journey_time_B_O[year] + GC_for_fuel_cost_B[year] / 100

user_benefit_A_non_work_low = {}
user_benefit_A_non_work_high = {}
user_benefit_B_non_work_low = {}
user_benefit_B_non_work_high = {}

for year in range(2031, 2091):
    user_benefit_A_non_work_low[year] = get_ROH(GC_A_non_work_fuel[year], GC_A_O_non_work_fuel[year], GC_O_non_work_fuel[year], 0.947 * AADT__A_dict_low[year], 0.947 * AADT__A_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    user_benefit_A_non_work_high[year] = get_ROH(GC_A_non_work_fuel[year], GC_A_O_non_work_fuel[year], GC_O_non_work_fuel[year], 0.947 * AADT__A_dict_high[year], 0.947 * AADT__A_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    user_benefit_B_non_work_low[year] = get_ROH(GC_B_non_work_fuel[year], GC_B_O_non_work_fuel[year], GC_O_non_work_fuel[year], 0.947 * AADT__B_dict_low[year], 0.947 * AADT__B_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    user_benefit_B_non_work_high[year] = get_ROH(GC_B_non_work_fuel[year], GC_B_O_non_work_fuel[year], GC_O_non_work_fuel[year], 0.947 * AADT__B_dict_high[year], 0.947 * AADT__B_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    
    

下面计算非工作非燃料的净收益

In [29]:
user_benefit_non_work_non_fuel_A_low = {}
user_benefit_non_work_non_fuel_A_high = {}
user_benefit_non_work_non_fuel_B_low = {}
user_benefit_non_work_non_fuel_B_high = {}

for year in range(2031, 2091):
    user_benefit_non_work_non_fuel_A_low[year] = (GC_for_non_fuel_non_Work_A / 100 *  0.947 * AADT__A_dict_low[year]  + GC_for_non_fuel_non_Work_A_O / 100 *  0.947 * AADT__A_O_dict_low[year] \
                                                - GC_for_non_fuel_non_Work_O / 100 *  0.947 * AADT__O_dict_low[year]) * 365.25 
    user_benefit_non_work_non_fuel_B_low[year] = (GC_for_non_fuel_non_Work_B / 100 *  0.947 * AADT__B_dict_low[year]  + GC_for_non_fuel_non_Work_B_O / 100 *  0.947 * AADT__B_O_dict_low[year] \
                                                - GC_for_non_fuel_non_Work_O / 100 *  0.947 * AADT__O_dict_low[year]) * 365.25
    user_benefit_non_work_non_fuel_A_high[year] = (GC_for_non_fuel_non_Work_A / 100 *  0.947 * AADT__A_dict_high[year]  + GC_for_non_fuel_non_Work_A_O / 100 *  0.947 * AADT__A_O_dict_high[year] \
                                                - GC_for_non_fuel_non_Work_O / 100 *  0.947 * AADT__O_dict_high[year]) * 365.25
    user_benefit_non_work_non_fuel_B_high[year] = (GC_for_non_fuel_non_Work_B / 100 *  0.947 * AADT__B_dict_high[year]  + GC_for_non_fuel_non_Work_B_O / 100 *  0.947 * AADT__B_O_dict_high[year] \
                                                - GC_for_non_fuel_non_Work_O / 100 *  0.947 * AADT__O_dict_high[year]) * 365.25



下面计算税收的损失，使用二分之一法则整合到前面计算的收益中（算作收益的损失而非成本）

In [30]:
t = 0.19
tf1 = 0
tf = 0.05
tn1 = 0
tn = 0.19
# formula in A.5
def calculate_tax(type, ROH_fuel, ROH_non_fuel):
    # 0 for non-work, 1 for work
    if type == 1:
        return ROH_fuel * tf1 * (1 + t) / (1 + tf1) + ROH_non_fuel * tn1 * (1 + t) / (1 + tn1)
    elif type == 0:
        return ROH_fuel * tf * (tf -t) / (1 + tf) + ROH_non_fuel * tn * (tn - t) / (1 + tn)
    




In [34]:
F_work_A_low = {}
F_work_A_high = {}
F_work_B_low = {}
F_work_B_high = {}
N_work_A_low ={}
N_work_A_high = {}
N_work_B_low = {}
N_work_B_high = {}
F_non_work_A_low = {}
F_non_work_A_high = {}
F_non_work_B_low = {}
F_non_work_B_high = {}
N_non_work_A_low = {}
N_non_work_A_high = {}
N_non_work_B_low = {}
N_non_work_B_high = {}
for year in range(2031, 2091):
    F_work_A_low[year] = get_ROH(GC_for_fuel_cost_A[year]/100, GC_for_fuel_cost_O[year]/100, GC_for_fuel_cost_O[year]/100, 0.053 * AADT__A_dict_low[year], 0.053 * AADT__A_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    F_work_A_high[year] = get_ROH(GC_for_fuel_cost_A[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.053 * AADT__A_dict_high[year], 0.053 * AADT__A_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    F_work_B_low[year] = get_ROH(GC_for_fuel_cost_B[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.053 * AADT__B_dict_low[year], 0.053 * AADT__B_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    F_work_B_high[year] = get_ROH(GC_for_fuel_cost_B[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.053 * AADT__B_dict_high[year], 0.053 * AADT__B_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    N_work_A_low[year] = get_ROH(GC_for_non_fuel_work_A, GC_for_non_fuel_work_A_O, GC_for_non_fuel_work_O, 0.053 * AADT__A_dict_low[year], 0.053 * AADT__A_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    N_work_A_high[year] = get_ROH(GC_for_non_fuel_work_A, GC_for_non_fuel_work_A_O, GC_for_non_fuel_work_O, 0.053 * AADT__A_dict_high[year], 0.053 * AADT__A_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    N_work_B_low[year] = get_ROH(GC_for_non_fuel_work_B, GC_for_non_fuel_work_B_O, GC_for_non_fuel_work_O, 0.053 * AADT__B_dict_low[year], 0.053 * AADT__B_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    N_work_B_high[year] = get_ROH(GC_for_non_fuel_work_B, GC_for_non_fuel_work_B_O, GC_for_non_fuel_work_O, 0.053 * AADT__B_dict_high[year], 0.053 * AADT__B_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    F_non_work_A_low[year] = get_ROH(GC_for_fuel_cost_A[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.947 * AADT__A_dict_low[year], 0.947 * AADT__A_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    F_non_work_A_high[year] = get_ROH(GC_for_fuel_cost_A[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.947 * AADT__A_dict_high[year], 0.947 * AADT__A_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    F_non_work_B_low[year] = get_ROH(GC_for_fuel_cost_B[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.947 * AADT__B_dict_low[year], 0.947 * AADT__B_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    F_non_work_B_high[year] = get_ROH(GC_for_fuel_cost_B[year], GC_for_fuel_cost_O[year], GC_for_fuel_cost_O[year], 0.947 * AADT__B_dict_high[year], 0.947 * AADT__B_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    N_non_work_A_low[year] = get_ROH(GC_for_non_fuel_non_Work_A, GC_for_non_fuel_non_Work_A_O, GC_for_non_fuel_non_Work_O, 0.947 * AADT__A_dict_low[year], 0.947 * AADT__A_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    N_non_work_A_high[year] = get_ROH(GC_for_non_fuel_non_Work_A, GC_for_non_fuel_non_Work_A_O, GC_for_non_fuel_non_Work_O, 0.947 * AADT__A_dict_high[year], 0.947 * AADT__A_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
    N_non_work_B_low[year] = get_ROH(GC_for_non_fuel_non_Work_B, GC_for_non_fuel_non_Work_B_O, GC_for_non_fuel_non_Work_O, 0.947 * AADT__B_dict_low[year], 0.947 * AADT__B_O_dict_low[year], 0.947 * AADT__O_dict_low[year])
    N_non_work_B_high[year] = get_ROH(GC_for_non_fuel_non_Work_B, GC_for_non_fuel_non_Work_B_O, GC_for_non_fuel_non_Work_O, 0.947 * AADT__B_dict_high[year], 0.947 * AADT__B_O_dict_high[year], 0.947 * AADT__O_dict_high[year])
print(GC_for_fuel_cost_A[2031]/100, GC_for_fuel_cost_O[2031]/100, GC_for_fuel_cost_O[2031]/100, 0.053 * AADT__A_dict_low[2031], 0.053 * AADT__A_O_dict_low[2031], 0.053 * AADT__O_dict_low[2031])
print(F_work_A_low[2031])

0.3895032719381687 0.21639070663231588 0.21639070663231588 689.0 212.0 802.9499999999999
-41194.581847909416


In [1363]:
# 计算间接税
indirect_tax_A_low = {}
indirect_tax_A_high = {}
indirect_tax_B_low = {}
indirect_tax_B_high = {}
for year in range(2031, 2091):
    indirect_tax_A_low[year] = calculate_tax(1, F_work_A_low[year], N_work_A_low[year]) + calculate_tax(0, F_non_work_A_low[year], N_non_work_A_low[year])
    indirect_tax_A_high[year] = calculate_tax(1, F_work_A_high[year], N_work_A_high[year]) + calculate_tax(0, F_non_work_A_high[year], N_non_work_A_high[year])
    indirect_tax_B_low[year] = calculate_tax(1, F_work_B_low[year], N_work_B_low[year]) + calculate_tax(0, F_non_work_B_low[year], N_non_work_B_low[year])
    indirect_tax_B_high[year] = calculate_tax(1, F_work_B_high[year], N_work_B_high[year]) + calculate_tax(0, F_non_work_B_high[year], N_non_work_B_high[year])

print(indirect_tax_A_low[2090])

505653.8344063382


下一步考虑温室气体效应，选用市场价格，仍然使用半数法算出效益；其实这里算作负效应还是正的支出，是一个值得讨论的话题；
假如改成汽油车，这一项估计要爆表了……

In [19]:

GC_emission_A = {}
GC_emission_B = {}
GC_emission_O = {}
for year in range(2031, 2091):
    GC_emission_A[year] = emission_factor[year] * money_value_of_emission[year] * fuel_effieciency[year] * road_length__A * 1.5893 / 1000
    GC_emission_B[year] = emission_factor[year] * money_value_of_emission[year] * fuel_effieciency[year] * road_length_B * 1.5893 / 1000
    GC_emission_O[year] = emission_factor[year] * money_value_of_emission[year] * fuel_effieciency[year] * road_length__O * 1.5893 / 1000

print(GC_emission_A[2090])
print(GC_emission_B[2031])
print(GC_emission_O[2031])
user_benefit_A_emission_low = {}
user_benefit_A_emission_high = {}
user_benefit_B_emission_low = {}
user_benefit_B_emission_high = {}
for year in range(2031, 2091):
    user_benefit_A_emission_low[year] = get_ROH(GC_emission_A[year], GC_emission_O[year], GC_emission_O[year],  AADT__A_dict_low[year],  AADT__A_O_dict_low[year],  AADT__O_dict_low[year])
    user_benefit_A_emission_high[year] = get_ROH(GC_emission_A[year], GC_emission_O[year], GC_emission_O[year],  AADT__A_dict_high[year],  AADT__A_O_dict_high[year],  AADT__O_dict_high[year])
    user_benefit_B_emission_low[year] = get_ROH(GC_emission_B[year], GC_emission_O[year], GC_emission_O[year],  AADT__B_dict_low[year],  AADT__B_O_dict_low[year],  AADT__O_dict_low[year])
    user_benefit_B_emission_high[year] = get_ROH(GC_emission_B[year], GC_emission_O[year], GC_emission_O[year],  AADT__B_dict_high[year],  AADT__B_O_dict_high[year],  AADT__O_dict_high[year])




0.00371226316503622
0.071334973276352
0.03242498785288728


NameError: name 'get_ROH' is not defined

最后一步，成本（建设和维护），我决定上面的都算是收益，哪怕是负收益

In [1365]:
maintainance_cost_A_2030 = maintainance_cost_A * 1.5893
maintainance_cost_B_2030 = maintenance_cost__B * 1.5893
maintainance_cost_A_dict = {}
maintainance_cost_B_dict = {}
for year in range(2031, 2091):
    maintainance_cost_A_dict[year] = maintainance_cost_A_2030
    maintainance_cost_B_dict[year] = maintainance_cost_B_2030

construction_cost__A_modified = construction_cost__A /2 * (1 + 1.035) * 1.5893
construction_cost__B_modified = construction_cost__B /2 * (1 + 1.035) * 1.5893



整合各部分成本和收益

In [1366]:
'''
1. 用半数法的部分
benefit_travel_time ... GC_for_journey_time
benefit_vehicle_operating_cost ... GC_for_fuel_cost, GC_for_non_fuel_work
benefit_emission ... GC_emission
GC_total_A_low = GC_for_journey_time_A + GC_for_fuel_cost_A + GC_emission_A + 0.053 * GC_for_non_fuel_non_work_A

2. 用复杂公式的部分，也用了半数法
benefit_indirect_tax ... indirect_tax_A, indirect_tax_B
3. 用净收益的部分
non_fuel_non_work ... GC_for_non_fuel_non_Work
'''
benefit_A_low = discount_to_2030(user_benefit_A_work_low, 2090) + discount_to_2030(user_benefit_A_non_work_low, 2090) + discount_to_2030(user_benefit_non_work_non_fuel_A_low, 2090) + discount_to_2030(user_benefit_A_emission_low, 2090) - discount_to_2030(indirect_tax_A_low, 2090)
benefit_A_high = discount_to_2030(user_benefit_A_work_high, 2090) + discount_to_2030(user_benefit_A_non_work_high, 2090) + discount_to_2030(user_benefit_non_work_non_fuel_A_high, 2090) + discount_to_2030(user_benefit_A_emission_high, 2090) - discount_to_2030(indirect_tax_A_high, 2090)
cost_A = construction_cost__A_modified + discount_to_2030(maintainance_cost_A_dict, 2090)
benefit_B_low = discount_to_2030(user_benefit_B_work_low, 2090) + discount_to_2030(user_benefit_B_non_work_low, 2090) + discount_to_2030(user_benefit_non_work_non_fuel_B_low, 2090) + discount_to_2030(user_benefit_B_emission_low, 2090) - discount_to_2030(indirect_tax_B_low, 2090)
benefit_B_high = discount_to_2030(user_benefit_B_work_high, 2090) + discount_to_2030(user_benefit_B_non_work_high, 2090) + discount_to_2030(user_benefit_non_work_non_fuel_B_high, 2090) + discount_to_2030(user_benefit_B_emission_high, 2090) - discount_to_2030(indirect_tax_B_high, 2090)
cost_B = construction_cost__B_modified + discount_to_2030(maintainance_cost_B_dict, 2090)
benefits_A_low = {}
benefits_A_high = {}
benefits_B_low = {}
benefits_B_high = {}
for year in range(2031, 2091):
    benefits_A_low[year] = discount_to_2030(user_benefit_A_work_low, year) + discount_to_2030(user_benefit_A_non_work_low, year) + discount_to_2030(user_benefit_non_work_non_fuel_A_low, year) + discount_to_2030(user_benefit_A_emission_low, year) - discount_to_2030(indirect_tax_A_low, year)
    benefits_A_high[year] = discount_to_2030(user_benefit_A_work_high, year) + discount_to_2030(user_benefit_A_non_work_high, year) + discount_to_2030(user_benefit_non_work_non_fuel_A_high, year) + discount_to_2030(user_benefit_A_emission_high, year) - discount_to_2030(indirect_tax_A_high, year)
    benefits_B_low[year] = discount_to_2030(user_benefit_B_work_low, year) + discount_to_2030(user_benefit_B_non_work_low, year) + discount_to_2030(user_benefit_non_work_non_fuel_B_low, year) + discount_to_2030(user_benefit_B_emission_low, year) - discount_to_2030(indirect_tax_B_low, year)
    benefits_B_high[year] = discount_to_2030(user_benefit_B_work_high, year) + discount_to_2030(user_benefit_B_non_work_high, year) + discount_to_2030(user_benefit_non_work_non_fuel_B_high, year) + discount_to_2030(user_benefit_B_emission_high, year) - discount_to_2030(indirect_tax_B_high, year)
    


PVB, PVC

In [1367]:
PVB_A_low = benefit_A_low
PVB_A_high = benefit_A_high
PVB_B_low = benefit_B_low
PVB_B_high = benefit_B_high
PVC_A_low = cost_A
PVC_A_high = cost_A
PVC_B_low = cost_B
PVC_B_high = cost_B

print(f"{'PVB_A_low:':>15}{PVB_A_low:>15.2f}")
print(f"{'PVB_A_high:':>15}{PVB_A_high:>15.2f}")
print(f"{'PVB_B_low:':>15}{PVB_B_low:>15.2f}")
print(f"{'PVB_B_high:':>15}{PVB_B_high:>15.2f}")
print(f"{'PVC_A_low:':>15}{PVC_A_low:>15.2f}")
print(f"{'PVC_A_high:':>15}{PVC_A_high:>15.2f}")
print(f"{'PVC_B_low:':>15}{PVC_B_low:>15.2f}")
print(f"{'PVC_B_high:':>15}{PVC_B_high:>15.2f}")




     PVB_A_low:   369347257.24
    PVB_A_high:   430324266.36
     PVB_B_low:   236784432.36
    PVB_B_high:   276350011.22
     PVC_A_low:   145943436.50
    PVC_A_high:   145943436.50
     PVC_B_low:   124920970.75
    PVC_B_high:   124920970.75


NPV

In [1368]:
NPV_A_low = benefit_A_low - cost_A
NPV_A_high = benefit_A_high - cost_A
NPV_B_low = benefit_B_low - cost_B
NPV_B_high = benefit_B_high - cost_B
# 格式化构成结果的各项和结果,数值靠右对齐
print(f"{'NPV_A_low:':>15}{NPV_A_low:>15.2f}")
print(f"{'NPV_A_high:':>15}{NPV_A_high:>15.2f}")
print(f"{'NPV_B_low:':>15}{NPV_B_low:>15.2f}")
print(f"{'NPV_B_high:':>15}{NPV_B_high:>15.2f}")


     NPV_A_low:   223403820.74
    NPV_A_high:   284380829.86
     NPV_B_low:   111863461.61
    NPV_B_high:   151429040.47


BCR

In [1369]:
BCR_A_low = benefit_A_low / cost_A
BCR_A_high = benefit_A_high / cost_A
BCR_B_low = benefit_B_low / cost_B
BCR_B_high = benefit_B_high / cost_B
print(f"{'BCR_A_low:':>15} {BCR_A_low:>15.2f}")
print(f"{'BCR_A_high:':>15} {BCR_A_high:>15.2f}")
print(f"{'BCR_B_low:':>15} {BCR_B_low:>15.2f}")
print(f"{'BCR_B_high:':>15} {BCR_B_high:>15.2f}")

     BCR_A_low:            2.53
    BCR_A_high:            2.95
     BCR_B_low:            1.90
    BCR_B_high:            2.21


IRR，使用numpy_financial包

In [1370]:
import numpy_financial as npf

cash_flows_A_low = [-construction_cost__A_modified] + [benefits_A_low[year] - maintainance_cost_A_dict[year] for year in range(2031, 2091)]
cash_flows_A_high = [-construction_cost__A_modified] + [benefits_A_high[year] - maintainance_cost_A_dict[year] for year in range(2031, 2091)]
cash_flows_B_low = [-construction_cost__B_modified] + [benefits_B_low[year] - maintainance_cost_B_dict[year] for year in range(2031, 2091)]
cash_flows_B_high = [-construction_cost__B_modified] + [benefits_B_high[year] - maintainance_cost_B_dict[year] for year in range(2031, 2091)]

irr_flows_A_low = npf.irr(cash_flows_A_low)
irr_flows_A_high = npf.irr(cash_flows_A_high)
irr_flows_B_low = npf.irr(cash_flows_B_low)
irr_flows_B_high = npf.irr(cash_flows_B_high)

print(f"{'IRR_A_low:':>15} {irr_flows_A_low:>15.2%}")
print(f"{'IRR_A_high:':>15} {irr_flows_A_high:>15.2%}")
print(f"{'IRR_B_low:':>15} {irr_flows_B_low:>15.2%}")
print(f"{'IRR_B_high:':>15} {irr_flows_B_high:>15.2%}")


     IRR_A_low:          26.44%
    IRR_A_high:          27.04%
     IRR_B_low:          21.68%
    IRR_B_high:          22.25%


Payback_Period

In [1371]:
def calculate_Payback_Period(benefits, costs, constant_cost):
    for year in range(2031, 2091):
        total_benefit = discount_to_2030(benefits, year)
        total_cost = discount_to_2030(costs, year) + constant_cost
        if total_benefit > total_cost:
            return year - 1

Payback_Period_A_low = calculate_Payback_Period(benefits_A_low, maintainance_cost_A_dict, construction_cost__A_modified)
Payback_Period_A_high = calculate_Payback_Period(benefits_A_high, maintainance_cost_A_dict, construction_cost__A_modified)
Payback_Period_B_low = calculate_Payback_Period(benefits_B_low, maintainance_cost_B_dict, construction_cost__B_modified)
Payback_Period_B_high = calculate_Payback_Period(benefits_B_high, maintainance_cost_B_dict, construction_cost__B_modified)
print('Payback_Period_A_low:', Payback_Period_A_low)
print('Payback_Period_A_high:', Payback_Period_A_high)
print('Payback_Period_B_low:', Payback_Period_B_low)
print('Payback_Period_B_high:', Payback_Period_B_high)

Payback_Period_A_low: 2035
Payback_Period_A_high: 2035
Payback_Period_B_low: 2037
Payback_Period_B_high: 2037


FYRR 


In [1372]:
benefit_A_low_2031 = benefits_A_low[2031]
benefit_A_high_2031 = benefits_A_high[2031]
benefit_B_low_2031 = benefits_B_low[2031]
benefit_B_high_2031 = benefits_B_high[2031]
cost_A_2031 = maintainance_cost_A_dict[2031] 
cost_B_2031 = maintainance_cost_B_dict[2031]
initial_cost_A = construction_cost__A_modified
initial_cost_B = construction_cost__B_modified
fyrr_A_low = (benefit_A_low_2031 - cost_A_2031) / initial_cost_A
fyrr_A_high = (benefit_A_high_2031 - cost_A_2031) / initial_cost_A
fyrr_B_low = (benefit_B_low_2031 - cost_B_2031) / initial_cost_B
fyrr_B_high = (benefit_B_high_2031 - cost_B_2031) / initial_cost_B
print('FYRR_A_low:', fyrr_A_low)
print('FYRR_A_high:', fyrr_A_high)
print('FYRR_B_low:', fyrr_B_low)
print('FYRR_B_high:', fyrr_B_high)


FYRR_A_low: 0.056410120346219166
FYRR_A_high: 0.05663008454105887
FYRR_B_low: 0.038518854060228376
FYRR_B_high: 0.038641369040337194


现在做一个总的汇总表，包含题目要求的各量：

1. PVC
2. PVB
3. NPV
4. BCR
5. IRR
6. FYRR
7. Pay_back_Period


In [1373]:
'''
现在做一个总的汇总表，包含题目要求的各量：

1. PVC
2. PVB
3. NPV
4. BCR
5. IRR
6. FYRR
7. Pay_back_Period
'''
import pandas as pd

data = {
    'A_low': [PVC_A_low, PVB_A_low, NPV_A_low, BCR_A_low, irr_flows_A_low, Payback_Period_A_low, fyrr_A_low],
    'A_high': [PVC_A_high, PVB_A_high, NPV_A_high, BCR_A_high, irr_flows_A_high, Payback_Period_A_high, fyrr_A_high],
    'B_low': [PVC_B_low, PVB_B_low, NPV_B_low, BCR_B_low, irr_flows_B_low, Payback_Period_B_low, fyrr_B_low],
    'B_high': [PVC_B_high, PVB_B_high, NPV_B_high, BCR_B_high, irr_flows_B_high, Payback_Period_B_high, fyrr_B_high]
}

df = pd.DataFrame(data, index=['PVC', 'PVB', 'NPV', 'BCR', 'IRR', 'Payback_Period', 'FYRR'])
print(df)
with pd.ExcelWriter('output_sum.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet1')

                       A_low        A_high         B_low        B_high
PVC             1.459434e+08  1.459434e+08  1.249210e+08  1.249210e+08
PVB             3.693473e+08  4.303243e+08  2.367844e+08  2.763500e+08
NPV             2.234038e+08  2.843808e+08  1.118635e+08  1.514290e+08
BCR             2.530756e+00  2.948569e+00  1.895474e+00  2.212199e+00
IRR             2.643930e-01  2.704123e-01  2.167644e-01  2.225044e-01
Payback_Period  2.035000e+03  2.035000e+03  2.037000e+03  2.037000e+03
FYRR            5.641012e-02  5.663008e-02  3.851885e-02  3.864137e-02


现在做一个汇总表，因为如果表格太大的话不好放入到文档中，因此，分几部分来输出:

1. VOC Benefit Data (这里包含了旅行时间，燃料成本和非燃料成本)
2. Emission Benefit Data
3. Indirect Tax Benefit Data
4. Cost Data

In [1374]:
import pandas as pd

# 初始化DataFrame
years = sorted(list(benefits_A_low.keys()))
df = pd.DataFrame(index=years)

# 用户利益数据（避免重复赋值，直接在创建时填充）
df_benefit_VOC = pd.DataFrame(index=df.index)
df_benefit_VOC['VOC_benefit_A_work_low'] = df.index.map(user_benefit_A_work_low)
df_benefit_VOC['VOC_benefit_A_work_high'] = df.index.map(user_benefit_A_work_high)
df_benefit_VOC['VOC_benefit_B_work_low'] = df.index.map(user_benefit_B_work_low)
df_benefit_VOC['VOC_benefit_B_work_high'] = df.index.map(user_benefit_B_work_high)
df_benefit_VOC['VOC_benefit_A_non_work_low'] = df.index.map(user_benefit_A_non_work_low)
df_benefit_VOC['VOC_benefit_A_non_work_high'] = df.index.map(user_benefit_A_non_work_high)
df_benefit_VOC['VOC_benefit_B_non_work_low'] = df.index.map(user_benefit_B_non_work_low)
df_benefit_VOC['VOC_benefit_B_non_work_high'] = df.index.map(user_benefit_B_non_work_high)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_A_low'] = df.index.map(user_benefit_non_work_non_fuel_A_low)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_A_high'] = df.index.map(user_benefit_non_work_non_fuel_A_high)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_B_low'] = df.index.map(user_benefit_non_work_non_fuel_B_low)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_B_high'] = df.index.map(user_benefit_non_work_non_fuel_B_high)
df_benefit_emission = pd.DataFrame(index=df.index)
df_benefit_emission['emission_benefit_A_low'] = df.index.map(user_benefit_A_emission_low)
df_benefit_emission['emission_benefit_A_high'] = df.index.map(user_benefit_A_emission_high)
df_benefit_emission['emission_benefit_B_low'] = df.index.map(user_benefit_B_emission_low)
df_benefit_emission['emission_benefit_B_high'] = df.index.map(user_benefit_B_emission_high)
df_benefit_indirect_tax = pd.DataFrame(index=df.index)
df_benefit_indirect_tax['indirect_tax_benefit_A_low'] = df.index.map(indirect_tax_A_low)
df_benefit_indirect_tax['indirect_tax_benefit_A_high'] = df.index.map(indirect_tax_A_high)
df_benefit_indirect_tax['indirect_tax_benefit_B_low'] = df.index.map(indirect_tax_B_low)
df_benefit_indirect_tax['indirect_tax_benefit_B_high'] = df.index.map(indirect_tax_B_high)


# 其他数据可以以类似的方式创建专门的DataFrame
df_cost = pd.DataFrame(index=df.index)
df_cost['construction_cost__A'] = construction_cost__A
df_cost['construction_cost__B'] = construction_cost__B 
df_cost['maintainance_cost_A'] = df.index.map(maintainance_cost_A_dict)
df_cost['maintainance_cost_B'] = df.index.map(maintainance_cost_B_dict)


# 注意：每个专门的DataFrame（例如df_AADT, df_GC, df_benefit等）可以根据需求单独保存或处理





In [1375]:
# Excel文件路径
excel_path = 'output_data.xlsx'

# 使用ExcelWriter将DataFrame写入不同的工作表
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='w') as writer:
    df_benefit_VOC.to_excel(writer, sheet_name='VOC Benefit Data', index=True)
    df_benefit_emission.to_excel(writer, sheet_name='Emission Benefit Data', index=True)
    df_benefit_indirect_tax.to_excel(writer, sheet_name='Indirect Tax Benefit Data', index=True)
    df_cost.to_excel(writer, sheet_name='Cost Data', index=True)

   


调整下列宽

In [1376]:
import pandas as pd
from openpyxl.utils import get_column_letter

excel_path = 'output_data.xlsx'
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='w') as writer:
    df_benefit_VOC.to_excel(writer, sheet_name='VOC Benefit Data', index=True)
    df_benefit_emission.to_excel(writer, sheet_name='Emission Benefit Data', index=True)
    df_benefit_indirect_tax.to_excel(writer, sheet_name='Indirect Tax Benefit Data', index=True)
    df_cost.to_excel(writer, sheet_name='Cost Data', index=True)

    workbook = writer.book
    worksheet = workbook['VOC Benefit Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40
    worksheet = workbook['Emission Benefit Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40
    worksheet = workbook['Indirect Tax Benefit Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40
    worksheet = workbook['Cost Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40

下面对construction cost, time value, emission, life span, discount进行敏感性测试

我个人认为该商业计划是一个OBC（Outline Business Case），这只是一个主观判断，因此 optimism_bias_uplifts 取值为 23%

根据说明，计算BCR即可，因此，在使用乐观偏差调整后得出BCR

同时，因为ppt中make a decision 在敏感性测试之后，所以测试内容还是得包括方案B....

先计算construction_cost

In [1377]:
optimism_bias_uplifts = 1.23
cost_A_uplift = construction_cost__A_modified * optimism_bias_uplifts + discount_to_2030(maintainance_cost_A_dict, 2090)
cost_B_uplift = construction_cost__B_modified * optimism_bias_uplifts + discount_to_2030(maintainance_cost_B_dict, 2090)

def BCR_calculator_by_cost_uplift(benefit, construction_cost, maintainance_cost, optimism_bias_uplifts):
    cost_uplift =   construction_cost * optimism_bias_uplifts + maintainance_cost
    return benefit / cost_uplift

'''
BCR_A_low = benefit_A_low / cost_A
BCR_A_high = benefit_A_high / cost_A
BCR_B_low = benefit_B_low / cost_B
BCR_B_high = benefit_B_high / cost_B
'''

BCR_A_low_uplift = BCR_calculator_by_cost_uplift(benefit_A_low, construction_cost__A_modified, discount_to_2030(maintainance_cost_A_dict, 2090), optimism_bias_uplifts)
BCR_A_high_uplift = BCR_calculator_by_cost_uplift(benefit_A_high, construction_cost__A_modified, discount_to_2030(maintainance_cost_A_dict, 2090), optimism_bias_uplifts)
BCR_B_low_uplift = BCR_calculator_by_cost_uplift(benefit_B_low, construction_cost__B_modified, discount_to_2030(maintainance_cost_B_dict, 2090), optimism_bias_uplifts)
BCR_B_high_uplift = BCR_calculator_by_cost_uplift(benefit_B_high, construction_cost__B_modified, discount_to_2030(maintainance_cost_B_dict, 2090), optimism_bias_uplifts)
print(f"{'BCR_A_low_uplift:':>15} {BCR_A_low_uplift:>15.2f}")
print(f"{'BCR_A_high_uplift:':>15} {BCR_A_high_uplift:>15.2f}")
print(f"{'BCR_B_low_uplift:':>15} {BCR_B_low_uplift:>15.2f}")
print(f"{'BCR_B_high_uplift:':>15} {BCR_B_high_uplift:>15.2f}")

def cost_needed_for_BCR(benefit, bcr, cost_uplift):
    """计算达到特定BCR所需的成本。"""
    return benefit / bcr - cost_uplift

def overrun_needed_for_BCR(benefit, bcr, cost):
    """计算达到特定BCR所需的成本超支比例。"""
    return benefit / bcr / cost - 1

cost_needed_for_BCR_2p5_low = cost_needed_for_BCR(benefit_A_low, 2.5, cost_A_uplift)
cost_needed_for_BCR_2p5_high = cost_needed_for_BCR(benefit_A_high, 2.5, cost_A_uplift)
cost_needed_for_BCR_2_low = cost_needed_for_BCR(benefit_A_low, 2, cost_A_uplift)
cost_needed_for_BCR_2_high = cost_needed_for_BCR(benefit_A_high, 2, cost_A_uplift)

cost_needed_for_B_for_BCR_2_low = cost_needed_for_BCR(benefit_B_low, 2, cost_B_uplift)
cost_needed_for_B_for_BCR_2_high = cost_needed_for_BCR(benefit_B_high, 2, cost_B_uplift)
cost_needed_for_B_for_BCR_1p5_low = cost_needed_for_BCR(benefit_B_low, 1.5, cost_B_uplift)
cost_needed_for_B_for_BCR_1p5_high = cost_needed_for_BCR(benefit_B_high, 1.5, cost_B_uplift)

overrun_needed_for_BCR_2p5_low = overrun_needed_for_BCR(benefit_A_low, 2.5, cost_A)
overrun_needed_for_BCR_2p5_high = overrun_needed_for_BCR(benefit_A_high, 2.5, cost_A)
overrun_needed_for_BCR_2_low = overrun_needed_for_BCR(benefit_A_low, 2, cost_A)
overrun_needed_for_BCR_2_high = overrun_needed_for_BCR(benefit_A_high, 2, cost_A)

overrun_needed_for_B_for_BCR_2_low = overrun_needed_for_BCR(benefit_B_low, 2, cost_B)
overrun_needed_for_B_for_BCR_2_high = overrun_needed_for_BCR(benefit_B_high, 2, cost_B)
overrun_needed_for_B_for_BCR_1p5_low = overrun_needed_for_BCR(benefit_B_low, 1.5, cost_B)
overrun_needed_for_B_for_BCR_1p5_high = overrun_needed_for_BCR(benefit_B_high, 1.5, cost_B)


vfm_data_1 = {
    "A_low: ": [cost_A, cost_A_uplift, benefit_A_low, BCR_A_low_uplift, cost_needed_for_BCR_2p5_low, cost_needed_for_BCR_2_low, overrun_needed_for_BCR_2p5_low, overrun_needed_for_BCR_2_low],
    "A_high: ": [cost_A, cost_A_uplift, benefit_A_high, BCR_A_high_uplift, cost_needed_for_BCR_2p5_high, cost_needed_for_BCR_2_high, overrun_needed_for_BCR_2p5_high, overrun_needed_for_BCR_2_high],
    "B_low:": [cost_B, cost_B_uplift, benefit_B_low, BCR_B_low_uplift, cost_needed_for_B_for_BCR_2_low, cost_needed_for_B_for_BCR_1p5_low, overrun_needed_for_B_for_BCR_2_low, overrun_needed_for_B_for_BCR_1p5_low],
    "B_high:": [cost_B, cost_B_uplift, benefit_B_high, BCR_B_high_uplift, cost_needed_for_B_for_BCR_2_high, cost_needed_for_B_for_BCR_1p5_high, overrun_needed_for_B_for_BCR_2_high, overrun_needed_for_B_for_BCR_1p5_high]
}

df = pd.DataFrame(vfm_data_1, index=['PVC', 'PVC_uplift', 'PVB', 'BCR_uplift', 'cost_needed_for_BCR_high', 'cost_needed_for_BCR_low', 'overrun_needed_for_BCR_high', 'overrun_needed_for_BCR_low'])
print(df)



BCR_A_low_uplift:            2.06
BCR_A_high_uplift:            2.40
BCR_B_low_uplift:            1.54
BCR_B_high_uplift:            1.80
                                  A_low:       A_high:         B_low:  \
PVC                          1.459434e+08  1.459434e+08  1.249210e+08   
PVC_uplift                   1.794177e+08  1.794177e+08  1.535600e+08   
PVB                          3.693473e+08  4.303243e+08  2.367844e+08   
BCR_uplift                   2.058589e+00  2.398450e+00  1.541966e+00   
cost_needed_for_BCR_high    -3.167877e+07 -7.287964e+06 -3.516782e+07   
cost_needed_for_BCR_low      5.255958e+06  3.574446e+07  4.296251e+06   
overrun_needed_for_BCR_high  1.230248e-02  1.794275e-01 -5.226308e-02   
overrun_needed_for_BCR_low   2.653781e-01  4.742844e-01  2.636492e-01   

                                  B_high:  
PVC                          1.249210e+08  
PVC_uplift                   1.535600e+08  
PVB                          2.763500e+08  
BCR_uplift                  

用非常牛逼的一个开源软件，[webplotdigitizer](https://automeris.io/documentation.html),生成了对应曲线提取的数据点，根据这些数据点线性插值

> 这个步骤是模仿tag 1.2教程中的示范，使用的是vfm评价图表

In [1378]:
import numpy as np

def linear_interpolation(x_values, y_values, target_y):
    """
    Performs linear interpolation to find the corresponding x value for a given target y value.

    Parameters:
    - x_values (list or array): The x values of the curve.
    - y_values (list or array): The y values of the curve.
    - target_y (float): The y value for which we want to find the corresponding x value.

    Returns:
    - target_x (float): The interpolated x value corresponding to the target y value.
    """
    if not (len(x_values) == len(y_values)):
        raise ValueError("x_values and y_values must have the same length")

    # Ensuring x_values and y_values are numpy arrays for efficient computation
    x_values = np.array(x_values)
    y_values = np.array(y_values)

    # Finding the first instance where the y value in the array exceeds the target y value
    for i in range(len(y_values) - 1):
        if y_values[i] <= target_y <= y_values[i + 1] or y_values[i] >= target_y >= y_values[i + 1]:
            # Performing linear interpolation
            slope = (x_values[i + 1] - x_values[i]) / (y_values[i + 1] - y_values[i])
            return x_values[i] + slope * (target_y - y_values[i])

    raise ValueError("target_y is outside the range of y_values")

# Reload the CSV with correct column interpretation
df = pd.read_csv('OBC_cost_road.csv', header=None, names=['x', 'y'])

# Convert percentage y values to decimals for interpolation
# -2对应的是高值，-1对应的是低值
y_target_values = [value[-2] for key, value in vfm_data_1.items()] + [value[-1] for key, value in vfm_data_1.items()]


# Use the linear interpolation function to find corresponding x values for the given y targets
x_interpolated = [linear_interpolation(df['x'].values, df['y'].values, y) for y in y_target_values]
x_interpolated


[0.19109492657337507,
 0.4443359866332839,
 0.131553412619402,
 0.3426140039368963,
 0.6424624113840605,
 0.7628817853178547,
 0.6420156945018313,
 0.7631477858399159]

继续计算关于项目周期的计算

In [1379]:

life_span = 60
life_span_corrected = int(60 / optimism_bias_uplifts)
life_span_corrected += 2030
print(life_span_corrected)
benefit_A_low_l_corrected = discount_to_2030(user_benefit_A_work_low, life_span_corrected) + discount_to_2030(user_benefit_A_non_work_low, life_span_corrected) + discount_to_2030(user_benefit_non_work_non_fuel_A_low, life_span_corrected) + discount_to_2030(user_benefit_A_emission_low, life_span_corrected) - discount_to_2030(indirect_tax_A_low, life_span_corrected)
benefit_A_high_l_corrected = discount_to_2030(user_benefit_A_work_high, life_span_corrected) + discount_to_2030(user_benefit_A_non_work_high, life_span_corrected) + discount_to_2030(user_benefit_non_work_non_fuel_A_high, life_span_corrected) + discount_to_2030(user_benefit_A_emission_high, life_span_corrected) - discount_to_2030(indirect_tax_A_high, life_span_corrected)
cost_A_l_corrected = construction_cost__A_modified + discount_to_2030(maintainance_cost_A_dict, life_span_corrected)
benefit_B_low_l_corrected = discount_to_2030(user_benefit_B_work_low, life_span_corrected) + discount_to_2030(user_benefit_B_non_work_low, life_span_corrected) + discount_to_2030(user_benefit_non_work_non_fuel_B_low, life_span_corrected) + discount_to_2030(user_benefit_B_emission_low, life_span_corrected) - discount_to_2030(indirect_tax_B_low, life_span_corrected)
benefit_B_high_l_corrected = discount_to_2030(user_benefit_B_work_high, life_span_corrected) + discount_to_2030(user_benefit_B_non_work_high, life_span_corrected) + discount_to_2030(user_benefit_non_work_non_fuel_B_high, life_span_corrected) + discount_to_2030(user_benefit_B_emission_high, life_span_corrected) - discount_to_2030(indirect_tax_B_high, life_span_corrected)
cost_B_l_corrected = construction_cost__B_modified + discount_to_2030(maintainance_cost_B_dict, life_span_corrected)

BCR_A_low_l_corrected = benefit_A_low_l_corrected / cost_A_l_corrected
BCR_A_high_l_corrected = benefit_A_high_l_corrected / cost_A_l_corrected
BCR_B_low_l_corrected = benefit_B_low_l_corrected / cost_B_l_corrected
BCR_B_high_l_corrected = benefit_B_high_l_corrected / cost_B_l_corrected
print(f"{'BCR_A_low_l_corrected:':>15} {BCR_A_low_l_corrected:>15.2f}")
print(f"{'BCR_A_high_l_corrected:':>15} {BCR_A_high_l_corrected:>15.2f}")
print(f"{'BCR_B_low_l_corrected:':>15} {BCR_B_low_l_corrected:>15.2f}")
print(f"{'BCR_B_high_l_corrected:':>15} {BCR_B_high_l_corrected:>15.2f}")
    



2078
BCR_A_low_l_corrected:            2.18
BCR_A_high_l_corrected:            2.52
BCR_B_low_l_corrected:            1.62
BCR_B_high_l_corrected:            1.88


时间价值，温室气体排放价值，和利率，分别改变23% 观测BCR的改变

实现途径可以是最简单的，复制几份程序，改变初始值，算出来结果即可……

1. discount_2 = discount * 1.23
2. value of time 2 = value of time / 1.23
3. emission_value (price) = emission_value (price) *1.23

因此，这部分内容见其他程序

如果所有电动汽车都变成了汽油车，会影响三部分：

1. 排放的影响，排放带来的负收益将非常严重
2. 燃油成本降低？
3. 间接税由损失转为正收入

一步步来考虑，先考虑排放的影响

In [1380]:
# litres
petrol_consumption_A = (0.45195 / average_speed__A + 0.09605 -0.00109 * average_speed__A + 0.000007 * average_speed__A ** 2) * road_length__A
petrol_consumption_B = (0.45195 / average_speed__B + 0.09605 -0.00109 * average_speed__B + 0.000007 * average_speed__B ** 2) * road_length_B
petrol_consumption_A_O = (0.45195 / average_speed__A_O + 0.09605 -0.00109 * average_speed__A_O + 0.000007 * average_speed__A_O ** 2) * road_length__O
petrol_consumption_B_O = (0.45195 / average_speed__B_O + 0.09605 -0.00109 * average_speed__B_O + 0.000007 * average_speed__B_O ** 2) * road_length__O
petrol_consumption_O = (0.45195 / average_speed__O + 0.09605 -0.00109 * average_speed__O + 0.000007 * average_speed__O ** 2) * road_length__O

petrol_kg_per_litre = 2.11

GC_petrol_emission_A = {}
GC_petrol_emission_B = {}
GC_petrol_emission_A_O = {}
GC_petrol_emission_B_O = {}
GC_petrol_emission_O = {}
# 除以1000是为了将kg转换为t
for year in range(2031, 2091):
    GC_petrol_emission_A[year] = petrol_consumption_A * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000
    GC_petrol_emission_B[year] = petrol_consumption_B * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000
    GC_petrol_emission_A_O[year] = petrol_consumption_A_O * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000
    GC_petrol_emission_B_O[year] = petrol_consumption_B_O * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000
    GC_petrol_emission_O[year] = petrol_consumption_O * petrol_kg_per_litre * money_value_of_emission[year] *1.5893 / 1000

user_benefit_A_emission_low_2 = {}
user_benefit_A_emission_high_2 = {}
user_benefit_B_emission_low_2 = {}
user_benefit_B_emission_high_2 = {}

for year in range(2031, 2091):
    user_benefit_A_emission_low_2[year] = get_ROH(GC_petrol_emission_A[year], GC_petrol_emission_A_O[year], GC_petrol_emission_O[year], AADT__A_dict_low[year],  AADT__A_O_dict_low[year],  AADT__O_dict_low[year])
    user_benefit_A_emission_high_2[year] = get_ROH(GC_petrol_emission_A[year], GC_petrol_emission_A_O[year], GC_petrol_emission_O[year], AADT__A_dict_high[year],  AADT__A_O_dict_high[year],  AADT__O_dict_high[year])
    user_benefit_B_emission_low_2[year] = get_ROH(GC_petrol_emission_B[year], GC_petrol_emission_B_O[year], GC_petrol_emission_O[year], AADT__B_dict_low[year],  AADT__B_O_dict_low[year],  AADT__O_dict_low[year])
    user_benefit_B_emission_high_2[year] = get_ROH(GC_petrol_emission_B[year], GC_petrol_emission_B_O[year], GC_petrol_emission_O[year], AADT__B_dict_high[year],  AADT__B_O_dict_high[year],  AADT__O_dict_high[year])


考虑燃料成本，工作的和非工作的

In [1381]:
# 这里包含旅行成本（注意命名可能不太合适）
GC_work_petrol_fuel_price_A = {}
GC_work_petrol_fuel_price_B = {}
GC_work_petrol_fuel_price_A_O = {}
GC_work_petrol_fuel_price_B_O = {}
GC_work_petrol_fuel_price_O = {}
# 除以100是讲便士转换为镑
for year in range(2031, 2091):
    GC_work_petrol_fuel_price_A[year] = petrol_fuel_price_65[year] * road_length__A /100 + GC_for_journey_time_A[year]
    GC_work_petrol_fuel_price_B[year] = petrol_fuel_price_65[year] * road_length_B /100 + GC_for_journey_time_B[year]
    GC_work_petrol_fuel_price_A_O[year] = petrol_fuel_price_28[year] * road_length__O /100 + GC_for_journey_time_A_O[year]
    GC_work_petrol_fuel_price_B_O[year] = petrol_fuel_price_28[year] * road_length__O /100 + GC_for_journey_time_B_O[year]
    GC_work_petrol_fuel_price_O[year] = petrol_fuel_price_22[year] * road_length__O /100 + GC_for_journey_time_O[year]

GC_non_work_petrol_fuel_price_A = {}
GC_non_work_petrol_fuel_price_B = {}
GC_non_work_petrol_fuel_price_A_O = {}
GC_non_work_petrol_fuel_price_B_O = {}
GC_non_work_petrol_fuel_price_O = {}
for year in range(2031, 2091):
    GC_non_work_petrol_fuel_price_A[year] = non_work_petrol_fuel_price_65[year] * road_length__A /100 + GC_for_journey_time_A[year]
    GC_non_work_petrol_fuel_price_B[year] = non_work_petrol_fuel_price_65[year] * road_length_B /100 + GC_for_journey_time_B[year]
    GC_non_work_petrol_fuel_price_A_O[year] = non_work_petrol_fuel_price_28[year] * road_length__O /100 + GC_for_journey_time_A_O[year]
    GC_non_work_petrol_fuel_price_B_O[year] = non_work_petrol_fuel_price_28[year] * road_length__O /100 + GC_for_journey_time_B_O[year]
    GC_non_work_petrol_fuel_price_O[year] = non_work_petrol_fuel_price_22[year] * road_length__O /100 + GC_for_journey_time_O[year]

user_benefit_A_work_low_2 = {}
user_benefit_A_work_high_2 = {}
user_benefit_B_work_low_2 = {}
user_benefit_B_work_high_2 = {}
user_benefit_A_non_work_low_2 = {}
user_benefit_A_non_work_high_2 = {}
user_benefit_B_non_work_low_2 = {}
user_benefit_B_non_work_high_2 = {}

for year in range(2031, 2091):
    user_benefit_A_work_low_2[year] = get_ROH(GC_work_petrol_fuel_price_A[year], GC_work_petrol_fuel_price_A_O[year], GC_work_petrol_fuel_price_O[year], 0.053 * AADT__A_dict_low[year],  0.053 * AADT__A_O_dict_low[year],  0.053 * AADT__O_dict_low[year])
    user_benefit_A_work_high_2[year] = get_ROH(GC_work_petrol_fuel_price_A[year], GC_work_petrol_fuel_price_A_O[year], GC_work_petrol_fuel_price_O[year], 0.053 * AADT__A_dict_high[year],  0.053 * AADT__A_O_dict_high[year],  0.053 * AADT__O_dict_high[year])
    user_benefit_B_work_low_2[year] = get_ROH(GC_work_petrol_fuel_price_B[year], GC_work_petrol_fuel_price_B_O[year], GC_work_petrol_fuel_price_O[year], 0.053 * AADT__B_dict_low[year],  0.053 * AADT__B_O_dict_low[year],  0.053 * AADT__O_dict_low[year])
    user_benefit_B_work_high_2[year] = get_ROH(GC_work_petrol_fuel_price_B[year], GC_work_petrol_fuel_price_B_O[year], GC_work_petrol_fuel_price_O[year], 0.053 * AADT__B_dict_high[year],  0.053 * AADT__B_O_dict_high[year],  0.053 * AADT__O_dict_high[year])
    user_benefit_A_non_work_low_2[year] = get_ROH(GC_non_work_petrol_fuel_price_A[year], GC_non_work_petrol_fuel_price_A_O[year], GC_non_work_petrol_fuel_price_O[year], 0.947 * AADT__A_dict_low[year],  0.947 * AADT__A_O_dict_low[year],  0.947 * AADT__O_dict_low[year])
    user_benefit_A_non_work_high_2[year] = get_ROH(GC_non_work_petrol_fuel_price_A[year], GC_non_work_petrol_fuel_price_A_O[year], GC_non_work_petrol_fuel_price_O[year], 0.947 * AADT__A_dict_high[year],  0.947 * AADT__A_O_dict_high[year],  0.947 * AADT__O_dict_high[year])
    user_benefit_B_non_work_low_2[year] = get_ROH(GC_non_work_petrol_fuel_price_B[year], GC_non_work_petrol_fuel_price_B_O[year], GC_non_work_petrol_fuel_price_O[year], 0.947 * AADT__B_dict_low[year],  0.947 * AADT__B_O_dict_low[year],  0.947 * AADT__O_dict_low[year])
    user_benefit_B_non_work_high_2[year] = get_ROH(GC_non_work_petrol_fuel_price_B[year], GC_non_work_petrol_fuel_price_B_O[year], GC_non_work_petrol_fuel_price_O[year], 0.947 * AADT__B_dict_high[year],  0.947 * AADT__B_O_dict_high[year],  0.947 * AADT__O_dict_high[year])

    
    

考虑非燃料成本，非燃料成本居然也和车的类型有关……

In [1382]:
C_65 = 4.966 + 135.946 / 65
C_28 = 4.966 + 135.946 / 28
C_22 = 4.966 + 135.946 / 22
GC_work_non_fuel_price_A = {}
GC_work_non_fuel_price_B = {}
GC_work_non_fuel_price_A_O = {}
GC_work_non_fuel_price_B_O = {}
GC_work_non_fuel_price_O = {}

for year in range(2031, 2091):
    GC_work_non_fuel_price_A[year] = C_65 * road_length__A /100 # 除以100是从便士转换为镑
    GC_work_non_fuel_price_B[year] = C_65 * road_length_B /100
    GC_work_non_fuel_price_A_O[year] = C_28 * road_length__O /100
    GC_work_non_fuel_price_B_O[year] = C_28 * road_length__O /100
    GC_work_non_fuel_price_O[year] = C_22 * road_length__O /100

user_benefit_work_non_fuel_A_low_2 = {}
user_benefit_work_non_fuel_A_high_2 = {}
user_benefit_work_non_fuel_B_low_2 = {}
user_benefit_work_non_fuel_B_high_2 = {}

for year in range(2031, 2091):
    user_benefit_work_non_fuel_A_low_2[year] = get_ROH(GC_work_non_fuel_price_A[year], GC_work_non_fuel_price_A_O[year], \
                                                       GC_work_non_fuel_price_O[year], 0.053 *AADT__A_dict_low[year],  \
                                                        0.053 *AADT__A_O_dict_low[year],  0.053 *AADT__O_dict_low[year])
    user_benefit_work_non_fuel_A_high_2[year] = get_ROH(GC_work_non_fuel_price_A[year], GC_work_non_fuel_price_A_O[year], \
                                                        GC_work_non_fuel_price_O[year], 0.053 *AADT__A_dict_high[year],  \
                                                            0.053 *AADT__A_O_dict_high[year],  0.053 *AADT__O_dict_high[year])
    user_benefit_work_non_fuel_B_low_2[year] = get_ROH(GC_work_non_fuel_price_B[year], GC_work_non_fuel_price_B_O[year], \
                                                       GC_work_non_fuel_price_O[year], 0.053 *AADT__B_dict_low[year],  \
                                                        0.053 *AADT__B_O_dict_low[year], 0.053 * AADT__O_dict_low[year])
    user_benefit_work_non_fuel_B_high_2[year] = get_ROH(GC_work_non_fuel_price_B[year], GC_work_non_fuel_price_B_O[year],\
                                                       GC_work_non_fuel_price_O[year], 0.053 *AADT__B_dict_high[year],  \
                                                          0.053 *  AADT__B_O_dict_high[year], 0.053 * AADT__O_dict_high[year])
    
C_2 = 3.846
GC_non_work_non_fuel_price_A = {}
GC_non_work_non_fuel_price_B = {}
GC_non_work_non_fuel_price_A_O = {}
GC_non_work_non_fuel_price_B_O = {}
GC_non_work_non_fuel_price_O = {}

for year in range(2031, 2091):
    GC_non_work_non_fuel_price_A[year] = C_2 * road_length__A /100
    GC_non_work_non_fuel_price_B[year] = C_2 * road_length_B /100
    GC_non_work_non_fuel_price_A_O[year] = C_2 * road_length__O /100
    GC_non_work_non_fuel_price_B_O[year] = C_2 * road_length__O /100
    GC_non_work_non_fuel_price_O[year] = C_2 * road_length__O /100

user_benefit_A_non_work_non_fuel_low_2 = {}
user_benefit_A_non_work_non_fuel_high_2 = {}
user_benefit_B_non_work_non_fuel_low_2 = {}
user_benefit_B_non_work_non_fuel_high_2 = {}

for year in range(2031, 2091):
    user_benefit_A_non_work_non_fuel_low_2[year] = (GC_non_work_non_fuel_price_A[year] * 0.947 * AADT__A_dict_low[year] + \
                                                    GC_non_work_non_fuel_price_A_O[year]\
                                                   * 0.947 * AADT__A_O_dict_low[year] - GC_non_work_non_fuel_price_O[year] * \
                                                    0.947 * AADT__O_dict_low[year]) * 365.25
    user_benefit_A_non_work_non_fuel_high_2[year] = (GC_non_work_non_fuel_price_A[year] * 0.947 * AADT__A_dict_high[year] + GC_non_work_non_fuel_price_A_O[year]\
                                                    * 0.947 * AADT__A_O_dict_high[year] - GC_non_work_non_fuel_price_O[year] \
                                                        * 0.947 * AADT__O_dict_high[year]) * 365.25
    user_benefit_B_non_work_non_fuel_low_2[year] = (GC_non_work_non_fuel_price_B[year] * 0.947 * AADT__B_dict_low[year] + \
                                                    GC_non_work_non_fuel_price_B_O[year]\
                                                    * 0.947 * AADT__B_O_dict_low[year] - GC_non_work_non_fuel_price_O[year] * \
                                                       0.947 *  AADT__O_dict_low[year]) * 365.25
    user_benefit_B_non_work_non_fuel_high_2[year] = (GC_non_work_non_fuel_price_B[year] * 0.947 * AADT__B_dict_high[year] + \
                                                     GC_non_work_non_fuel_price_B_O[year]\
                                                     * 0.947 * AADT__B_O_dict_high[year] - GC_non_work_non_fuel_price_O[year] * \
                                                        0.947 * AADT__O_dict_high[year]) * 365.25
    


考虑间接税

In [1383]:
tf = 0.2
# 计算间接税
indirect_tax_A_low_2 = {}
indirect_tax_A_high_2 = {}
indirect_tax_B_low_2 = {}
indirect_tax_B_high_2 = {}

F_work_A_low_2 = {}
F_work_A_high_2 = {}
F_work_B_low_2 = {}
F_work_B_high_2 = {}
N_work_A_low_2 ={}
N_work_A_high_2 = {}
N_work_B_low_2 = {}
N_work_B_high_2 = {}
F_non_work_A_low_2 = {}
F_non_work_A_high_2 = {}
F_non_work_B_low_2 = {}
F_non_work_B_high_2 = {}
N_non_work_A_low_2 = {}
N_non_work_A_high_2 = {}
N_non_work_B_low_2 = {}
N_non_work_B_high_2 = {}

for year in range(2031, 2091):
    F_work_A_low_2[year] = get_ROH(GC_work_petrol_fuel_price_A[year], \
                                GC_work_petrol_fuel_price_A_O[year], \
                                GC_work_petrol_fuel_price_O[year], \
                                0.053 * AADT__A_dict_low[year],  \
                                0.053 * AADT__A_O_dict_low[year],  \
                                0.053 * AADT__O_dict_low[year])

    F_work_A_high_2[year] = get_ROH(GC_work_petrol_fuel_price_A[year], \
                                    GC_work_petrol_fuel_price_A_O[year], \
                                    GC_work_petrol_fuel_price_O[year], \
                                    0.053 * AADT__A_dict_high[year],  \
                                    0.053 * AADT__A_O_dict_high[year],  \
                                    0.053 * AADT__O_dict_high[year])

    F_work_B_low_2[year] = get_ROH(GC_work_petrol_fuel_price_B[year], \
                                    GC_work_petrol_fuel_price_B_O[year], \
                                    GC_work_petrol_fuel_price_O[year], \
                                    0.053 * AADT__B_dict_low[year],  \
                                    0.053 * AADT__B_O_dict_low[year],  \
                                    0.053 * AADT__O_dict_low[year])

    F_work_B_high_2[year] = get_ROH(GC_work_petrol_fuel_price_B[year], \
                                    GC_work_petrol_fuel_price_B_O[year], \
                                    GC_work_petrol_fuel_price_O[year], \
                                    0.053 * AADT__B_dict_high[year],  \
                                    0.053 * AADT__B_O_dict_high[year],  \
                                    0.053 * AADT__O_dict_high[year])

    N_work_A_low_2[year] = get_ROH(GC_work_non_fuel_price_A[year], \
                                    GC_work_non_fuel_price_A_O[year], \
                                    GC_work_non_fuel_price_O[year], \
                                    0.053 * AADT__A_dict_low[year],  \
                                    0.053 * AADT__A_O_dict_low[year],  \
                                    0.053 * AADT__O_dict_low[year])

    N_work_A_high_2[year] = get_ROH(GC_work_non_fuel_price_A[year], \
                                    GC_work_non_fuel_price_A_O[year], \
                                    GC_work_non_fuel_price_O[year], \
                                    0.053 * AADT__A_dict_high[year],  \
                                    0.053 * AADT__A_O_dict_high[year],  \
                                    0.053 * AADT__O_dict_high[year])

    N_work_B_low_2[year] = get_ROH(GC_work_non_fuel_price_B[year], \
                                    GC_work_non_fuel_price_B_O[year], \
                                    GC_work_non_fuel_price_O[year], \
                                    0.053 * AADT__B_dict_low[year],  \
                                    0.053 * AADT__B_O_dict_low[year],  \
                                    0.053 * AADT__O_dict_low[year])

    N_work_B_high_2[year] = get_ROH(GC_work_non_fuel_price_B[year], \
                                    GC_work_non_fuel_price_B_O[year], \
                                    GC_work_non_fuel_price_O[year], \
                                    0.053 * AADT__B_dict_high[year],  \
                                    0.053 * AADT__B_O_dict_high[year],  \
                                    0.053 * AADT__O_dict_high[year])

    F_non_work_A_low_2[year] = get_ROH(GC_non_work_petrol_fuel_price_A[year], \
                                        GC_non_work_petrol_fuel_price_A_O[year], \
                                        GC_non_work_petrol_fuel_price_O[year], \
                                        0.947 * AADT__A_dict_low[year],  \
                                        0.947 * AADT__A_O_dict_low[year],  \
                                        0.947 * AADT__O_dict_low[year])

    F_non_work_A_high_2[year] = get_ROH(GC_non_work_petrol_fuel_price_A[year], \
                                        GC_non_work_petrol_fuel_price_A_O[year], \
                                        GC_non_work_petrol_fuel_price_O[year], \
                                        0.947 * AADT__A_dict_high[year],  \
                                        0.947 * AADT__A_O_dict_high[year],  \
                                        0.947 * AADT__O_dict_high[year])

    F_non_work_B_low_2[year] = get_ROH(GC_non_work_petrol_fuel_price_B[year], \
                                        GC_non_work_petrol_fuel_price_B_O[year], \
                                        GC_non_work_petrol_fuel_price_O[year], \
                                        0.947 * AADT__B_dict_low[year],  \
                                        0.947 * AADT__B_O_dict_low[year],  \
                                        0.947 * AADT__O_dict_low[year])

    F_non_work_B_high_2[year] = get_ROH(GC_non_work_petrol_fuel_price_B[year], \
                                        GC_non_work_petrol_fuel_price_B_O[year], \
                                        GC_non_work_petrol_fuel_price_O[year], \
                                        0.947 * AADT__B_dict_high[year],  \
                                        0.947 * AADT__B_O_dict_high[year],  \
                                        0.947 * AADT__O_dict_high[year])

    N_non_work_A_low_2[year] = get_ROH(GC_non_work_non_fuel_price_A[year], \
                                        GC_non_work_non_fuel_price_A_O[year], \
                                        GC_non_work_non_fuel_price_O[year], \
                                        0.947 * AADT__A_dict_low[year],  \
                                        0.947 * AADT__A_O_dict_low[year],  \
                                        0.947 * AADT__O_dict_low[year])

    N_non_work_A_high_2[year] = get_ROH(GC_non_work_non_fuel_price_A[year], \
                                        GC_non_work_non_fuel_price_A_O[year], \
                                        GC_non_work_non_fuel_price_O[year], \
                                        0.947 * AADT__A_dict_high[year],  \
                                        0.947 * AADT__A_O_dict_high[year],  \
                                        0.947 * AADT__O_dict_high[year])

    N_non_work_B_low_2[year] = get_ROH(GC_non_work_non_fuel_price_B[year], \
                                        GC_non_work_non_fuel_price_B_O[year], \
                                        GC_non_work_non_fuel_price_O[year], \
                                        0.947 * AADT__B_dict_low[year],  \
                                        0.947 * AADT__B_O_dict_low[year],  \
                                        0.947 * AADT__O_dict_low[year])

    N_non_work_B_high_2[year] = get_ROH(GC_non_work_non_fuel_price_B[year], \
                                        GC_non_work_non_fuel_price_B_O[year], \
                                        GC_non_work_non_fuel_price_O[year], \
                                        0.947 * AADT__B_dict_high[year],  \
                                        0.947 * AADT__B_O_dict_high[year],  \
                                        0.947 * AADT__O_dict_high[year])

for year in range(2031, 2091):
    indirect_tax_A_low_2[year] = calculate_tax(1, F_work_A_low_2[year], N_work_A_low_2[year]) + calculate_tax(0, F_non_work_A_low_2[year], N_non_work_A_low_2[year])
    indirect_tax_A_high_2[year] = calculate_tax(1, F_work_A_high_2[year], N_work_A_high_2[year]) + calculate_tax(0, F_non_work_A_high_2[year], N_non_work_A_high_2[year])
    indirect_tax_B_low_2[year] = calculate_tax(1, F_work_B_low_2[year], N_work_B_low_2[year]) + calculate_tax(0, F_non_work_B_low_2[year], N_non_work_B_low_2[year])
    indirect_tax_B_high_2[year] = calculate_tax(1, F_work_B_high_2[year], N_work_B_high_2[year]) + calculate_tax(0, F_non_work_B_high_2[year], N_non_work_B_high_2[year])

print(indirect_tax_A_low_2[2090])

43885.48718003852


最后整合起来

In [1384]:
benefit_A_low_2 = discount_to_2030(user_benefit_A_work_low_2, 2090) + discount_to_2030(user_benefit_A_non_work_low_2, 2090) \
    + discount_to_2030(user_benefit_A_emission_low_2, 2090) + discount_to_2030(user_benefit_A_non_work_non_fuel_low_2, 2090) \
        - discount_to_2030(indirect_tax_A_low_2, 2090)
benefit_A_high_2 = discount_to_2030(user_benefit_A_work_high_2, 2090) + discount_to_2030(user_benefit_A_non_work_high_2, 2090) \
    + discount_to_2030(user_benefit_A_emission_high_2, 2090) + discount_to_2030(user_benefit_A_non_work_non_fuel_high_2, 2090) \
        - discount_to_2030(indirect_tax_A_high_2, 2090)
cost_A_2 = construction_cost__A_modified + discount_to_2030(maintainance_cost_A_dict, 2090)
benefit_B_low_2 = discount_to_2030(user_benefit_B_work_low_2, 2090) + discount_to_2030(user_benefit_B_non_work_low_2, 2090) \
    + discount_to_2030(user_benefit_B_emission_low_2, 2090) + discount_to_2030(user_benefit_B_non_work_non_fuel_low_2, 2090) \
        - discount_to_2030(indirect_tax_B_low_2, 2090)
benefit_B_high_2 = discount_to_2030(user_benefit_B_work_high_2, 2090) + discount_to_2030(user_benefit_B_non_work_high_2, 2090) \
    + discount_to_2030(user_benefit_B_emission_high_2, 2090) + discount_to_2030(user_benefit_B_non_work_non_fuel_high_2, 2090) \
        - discount_to_2030(indirect_tax_B_high_2, 2090)
cost_B_2 = construction_cost__B_modified + discount_to_2030(maintainance_cost_B_dict, 2090)

benefits_A_low_2 = {}
benefits_A_high_2 = {}
benefits_B_low_2 = {}
benefits_B_high_2 = {}

for year in range(2031, 2091):
    benefits_A_low_2[year] = discount_to_2030(user_benefit_A_work_low_2, year) + discount_to_2030(user_benefit_A_non_work_low_2, year) \
    + discount_to_2030(user_benefit_A_emission_low_2, year) + discount_to_2030(user_benefit_A_non_work_non_fuel_low_2, year) \
        - discount_to_2030(indirect_tax_A_low_2, year)
    benefits_A_high_2[year] = discount_to_2030(user_benefit_A_work_high_2, year) + discount_to_2030(user_benefit_A_non_work_high_2, year) \
    + discount_to_2030(user_benefit_A_emission_high_2, year) + discount_to_2030(user_benefit_A_non_work_non_fuel_high_2, year) \
        - discount_to_2030(indirect_tax_A_high_2, year)
    benefits_B_low_2[year] = discount_to_2030(user_benefit_B_work_low_2, year) + discount_to_2030(user_benefit_B_non_work_low_2, year) \
    + discount_to_2030(user_benefit_B_emission_low_2, year) + discount_to_2030(user_benefit_B_non_work_non_fuel_low_2, year) \
        - discount_to_2030(indirect_tax_B_low_2, year)
    benefits_B_high_2[year] = discount_to_2030(user_benefit_B_work_high_2, year) + discount_to_2030(user_benefit_B_non_work_high_2, year) \
    + discount_to_2030(user_benefit_B_emission_high_2, year) + discount_to_2030(user_benefit_B_non_work_non_fuel_high_2, year) \
        - discount_to_2030(indirect_tax_B_high_2, year)
    



In [1385]:
PVB_A_low = benefit_A_low_2
PVB_A_high = benefit_A_high_2
PVB_B_low = benefit_B_low_2
PVB_B_high = benefit_B_high_2
PVC_A_low = cost_A_2
PVC_A_high = cost_A_2
PVC_B_low = cost_B_2
PVC_B_high = cost_B_2

print(f"{PVB_A_low}, {PVB_A_high}, {PVB_B_low}, {PVB_B_high}")
print(f"{PVC_A_low}, {PVC_A_high}, {PVC_B_low}, {PVC_B_high}")

398832021.5902845, 463597740.9574793, 250301467.6481173, 290796212.12253433
145943436.49927253, 145943436.49927253, 124920970.74927253, 124920970.74927253


In [1386]:
NPV_A_low = benefit_A_low_2 - cost_A_2
NPV_A_high = benefit_A_high_2 - cost_A_2
NPV_B_low = benefit_B_low_2 - cost_B_2
NPV_B_high = benefit_B_high_2 - cost_B_2
# 格式化构成结果的各项和结果,数值靠右对齐
print(f"{'NPV_A_low:':>15}{NPV_A_low:>15.2f}")
print(f"{'NPV_A_high:':>15}{NPV_A_high:>15.2f}")
print(f"{'NPV_B_low:':>15}{NPV_B_low:>15.2f}")
print(f"{'NPV_B_high:':>15}{NPV_B_high:>15.2f}")

     NPV_A_low:   252888585.09
    NPV_A_high:   317654304.46
     NPV_B_low:   125380496.90
    NPV_B_high:   165875241.37


In [1387]:
BCR_A_low = benefit_A_low_2 / cost_A_2
BCR_A_high = benefit_A_high_2 / cost_A_2
BCR_B_low = benefit_B_low_2 / cost_B_2
BCR_B_high = benefit_B_high_2 / cost_B_2
print(f"{'BCR_A_low:':>15} {BCR_A_low:>15.2f}")
print(f"{'BCR_A_high:':>15} {BCR_A_high:>15.2f}")
print(f"{'BCR_B_low:':>15} {BCR_B_low:>15.2f}")
print(f"{'BCR_B_high:':>15} {BCR_B_high:>15.2f}")

     BCR_A_low:            2.73
    BCR_A_high:            3.18
     BCR_B_low:            2.00
    BCR_B_high:            2.33


In [1388]:
import numpy_financial as npf

cash_flows_A_low = [-construction_cost__A_modified] + [benefits_A_low_2[year] - maintainance_cost_A_dict[year] for year in range(2031, 2091)]
cash_flows_A_high = [-construction_cost__A_modified] + [benefits_A_high_2[year] - maintainance_cost_A_dict[year] for year in range(2031, 2091)]
cash_flows_B_low = [-construction_cost__B_modified] + [benefits_B_low_2[year] - maintainance_cost_B_dict[year] for year in range(2031, 2091)]
cash_flows_B_high = [-construction_cost__B_modified] + [benefits_B_high_2[year] - maintainance_cost_B_dict[year] for year in range(2031, 2091)]

irr_flows_A_low = npf.irr(cash_flows_A_low)
irr_flows_A_high = npf.irr(cash_flows_A_high)
irr_flows_B_low = npf.irr(cash_flows_B_low)
irr_flows_B_high = npf.irr(cash_flows_B_high)

print(f"{'IRR_A_low:':>15} {irr_flows_A_low:>15.2%}")
print(f"{'IRR_A_high:':>15} {irr_flows_A_high:>15.2%}")
print(f"{'IRR_B_low:':>15} {irr_flows_B_low:>15.2%}")
print(f"{'IRR_B_high:':>15} {irr_flows_B_high:>15.2%}")

     IRR_A_low:          28.53%
    IRR_A_high:          29.13%
     IRR_B_low:          23.82%
    IRR_B_high:          24.38%


In [1389]:
def calculate_Payback_Period(benefits, costs, constant_cost):
    for year in range(2031, 2091):
        total_benefit = discount_to_2030(benefits, year)
        total_cost = discount_to_2030(costs, year) + constant_cost
        if total_benefit > total_cost:
            return year - 1

Payback_Period_A_low = calculate_Payback_Period(benefits_A_low_2, maintainance_cost_A_dict, construction_cost__A_modified)
Payback_Period_A_high = calculate_Payback_Period(benefits_A_high_2, maintainance_cost_A_dict, construction_cost__A_modified)
Payback_Period_B_low = calculate_Payback_Period(benefits_B_low_2, maintainance_cost_B_dict, construction_cost__B_modified)
Payback_Period_B_high = calculate_Payback_Period(benefits_B_high_2, maintainance_cost_B_dict, construction_cost__B_modified)
print('Payback_Period_A_low:', Payback_Period_A_low)
print('Payback_Period_A_high:', Payback_Period_A_high)
print('Payback_Period_B_low:', Payback_Period_B_low)
print('Payback_Period_B_high:', Payback_Period_B_high)

Payback_Period_A_low: 2035
Payback_Period_A_high: 2035
Payback_Period_B_low: 2036
Payback_Period_B_high: 2036


In [1390]:
benefit_A_low_2031 = benefits_A_low_2[2031]
benefit_A_high_2031 = benefits_A_high_2[2031]
benefit_B_low_2031 = benefits_B_low_2[2031]
benefit_B_high_2031 = benefits_B_high_2[2031]
cost_A_2031 = maintainance_cost_A_dict[2031] 
cost_B_2031 = maintainance_cost_B_dict[2031]
initial_cost_A = construction_cost__A_modified
initial_cost_B = construction_cost__B_modified
fyrr_A_low = (benefit_A_low_2031 - cost_A_2031) / initial_cost_A
fyrr_A_high = (benefit_A_high_2031 - cost_A_2031) / initial_cost_A
fyrr_B_low = (benefit_B_low_2031 - cost_B_2031) / initial_cost_B
fyrr_B_high = (benefit_B_high_2031 - cost_B_2031) / initial_cost_B
print('FYRR_A_low:', fyrr_A_low)
print('FYRR_A_high:', fyrr_A_high)
print('FYRR_B_low:', fyrr_B_low)
print('FYRR_B_high:', fyrr_B_high)

FYRR_A_low: 0.065576868157415
FYRR_A_high: 0.06579186274311667
FYRR_B_low: 0.0478619717031628
FYRR_B_high: 0.04796499132027147


In [1391]:
'''
现在做一个总的汇总表，包含题目要求的各量：

1. PVC
2. PVB
3. NPV
4. BCR
5. IRR
6. FYRR
7. Pay_back_Period
'''
import pandas as pd

data = {
    'A_low': [PVC_A_low, PVB_A_low, NPV_A_low, BCR_A_low, irr_flows_A_low, Payback_Period_A_low, fyrr_A_low],
    'A_high': [PVC_A_high, PVB_A_high, NPV_A_high, BCR_A_high, irr_flows_A_high, Payback_Period_A_high, fyrr_A_high],
    'B_low': [PVC_B_low, PVB_B_low, NPV_B_low, BCR_B_low, irr_flows_B_low, Payback_Period_B_low, fyrr_B_low],
    'B_high': [PVC_B_high, PVB_B_high, NPV_B_high, BCR_B_high, irr_flows_B_high, Payback_Period_B_high, fyrr_B_high]
}

df = pd.DataFrame(data, index=['PVC', 'PVB', 'NPV', 'BCR', 'IRR', 'Payback_Period', 'FYRR'])
print(df)
with pd.ExcelWriter('output_sum_2.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet1')

                       A_low        A_high         B_low        B_high
PVC             1.459434e+08  1.459434e+08  1.249210e+08  1.249210e+08
PVB             3.988320e+08  4.635977e+08  2.503015e+08  2.907962e+08
NPV             2.528886e+08  3.176543e+08  1.253805e+08  1.658752e+08
BCR             2.732785e+00  3.176558e+00  2.003679e+00  2.327841e+00
IRR             2.853156e-01  2.912928e-01  2.382377e-01  2.438347e-01
Payback_Period  2.035000e+03  2.035000e+03  2.036000e+03  2.036000e+03
FYRR            6.557687e-02  6.579186e-02  4.786197e-02  4.796499e-02


In [1392]:
import pandas as pd

# 初始化DataFrame
years = sorted(list(benefits_A_low.keys()))
df = pd.DataFrame(index=years)

# 用户利益数据（避免重复赋值，直接在创建时填充）
df_benefit_VOC = pd.DataFrame(index=df.index)
df_benefit_VOC['VOC_benefit_A_work_low'] = df.index.map(user_benefit_A_work_low_2)
df_benefit_VOC['VOC_benefit_A_work_high'] = df.index.map(user_benefit_A_work_high_2)
df_benefit_VOC['VOC_benefit_B_work_low'] = df.index.map(user_benefit_B_work_low_2)
df_benefit_VOC['VOC_benefit_B_work_high'] = df.index.map(user_benefit_B_work_high_2)
df_benefit_VOC['VOC_benefit_A_non_work_low'] = df.index.map(user_benefit_A_non_work_low_2)
df_benefit_VOC['VOC_benefit_A_non_work_high'] = df.index.map(user_benefit_A_non_work_high_2)
df_benefit_VOC['VOC_benefit_B_non_work_low'] = df.index.map(user_benefit_B_non_work_low_2)
df_benefit_VOC['VOC_benefit_B_non_work_high'] = df.index.map(user_benefit_B_non_work_high_2)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_A_low'] = df.index.map(user_benefit_A_non_work_non_fuel_low_2)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_A_high'] = df.index.map(user_benefit_A_non_work_non_fuel_high_2)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_B_low'] = df.index.map(user_benefit_B_non_work_non_fuel_low_2)
df_benefit_VOC['VOC_beneift_non_fuel_non_work_B_high'] = df.index.map(user_benefit_B_non_work_non_fuel_high_2)
df_benefit_emission = pd.DataFrame(index=df.index)
df_benefit_emission['emission_benefit_A_low'] = df.index.map(user_benefit_A_emission_low_2)
df_benefit_emission['emission_benefit_A_high'] = df.index.map(user_benefit_A_emission_high_2)
df_benefit_emission['emission_benefit_B_low'] = df.index.map(user_benefit_B_emission_low_2)
df_benefit_emission['emission_benefit_B_high'] = df.index.map(user_benefit_B_emission_high_2)
df_benefit_indirect_tax = pd.DataFrame(index=df.index)
df_benefit_indirect_tax['indirect_tax_benefit_A_low'] = df.index.map(indirect_tax_A_low_2)
df_benefit_indirect_tax['indirect_tax_benefit_A_high'] = df.index.map(indirect_tax_A_high_2)
df_benefit_indirect_tax['indirect_tax_benefit_B_low'] = df.index.map(indirect_tax_B_low_2)
df_benefit_indirect_tax['indirect_tax_benefit_B_high'] = df.index.map(indirect_tax_B_high_2)


# 其他数据可以以类似的方式创建专门的DataFrame
df_cost = pd.DataFrame(index=df.index)
df_cost['construction_cost__A'] = construction_cost__A_modified
df_cost['construction_cost__B'] = construction_cost__B_modified 
df_cost['maintainance_cost_A'] = df.index.map(maintainance_cost_A_dict)
df_cost['maintainance_cost_B'] = df.index.map(maintainance_cost_B_dict)


# 注意：每个专门的DataFrame（例如df_AADT, df_GC, df_benefit等）可以根据需求单独保存或处理

In [1393]:
# Excel文件路径
excel_path = 'output_data_2.xlsx'

# 使用ExcelWriter将DataFrame写入不同的工作表
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='w') as writer:
    df_benefit_VOC.to_excel(writer, sheet_name='VOC Benefit Data', index=True)
    df_benefit_emission.to_excel(writer, sheet_name='Emission Benefit Data', index=True)
    df_benefit_indirect_tax.to_excel(writer, sheet_name='Indirect Tax Benefit Data', index=True)
    df_cost.to_excel(writer, sheet_name='Cost Data', index=True)

In [1394]:
import pandas as pd
from openpyxl.utils import get_column_letter

excel_path = 'output_data_2.xlsx'
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='w') as writer:
    df_benefit_VOC.to_excel(writer, sheet_name='VOC Benefit Data', index=True)
    df_benefit_emission.to_excel(writer, sheet_name='Emission Benefit Data', index=True)
    df_benefit_indirect_tax.to_excel(writer, sheet_name='Indirect Tax Benefit Data', index=True)
    df_cost.to_excel(writer, sheet_name='Cost Data', index=True)

    workbook = writer.book
    worksheet = workbook['VOC Benefit Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40
    worksheet = workbook['Emission Benefit Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40
    worksheet = workbook['Indirect Tax Benefit Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40
    worksheet = workbook['Cost Data']
    for col in worksheet.columns:
        worksheet.column_dimensions[get_column_letter(col[0].column)].width = 40